In [1]:
import matplotlib.pyplot as plt
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="5"

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer, Normalizer
from sklearn.model_selection import StratifiedKFold
import tensorflow as tf
import pickle
import numpy as np
import pandas as pd

from annsa.template_sampling import *

from annsa.load_dataset import load_easy

In [2]:
import tensorflow.contrib.eager as tfe

In [3]:
tf.enable_eager_execution()

#### Import model, training function 

In [4]:
from annsa.model_classes import (dae_model_features,
                                 DAE,
                                 save_model,
                                 train_earlystop)

## Load testing dataset 

In [5]:
dataset = np.load('../dataset_generation/testing_dataset_full_200keV_log10time_1000.npy')

In [6]:
all_spectra = np.random.poisson(np.add(dataset.item()['sources'], dataset.item()['backgrounds']))
all_keys = np.array(dataset.item()['sources']) # background subtracted output
all_keys_modified = np.log1p(all_keys)/np.max(np.log1p(all_spectra),axis=1)[:,None]


keys_raw = dataset.item()['keys'] # keys necessary for stratified k-folds


## Train network

In [11]:
model_id = 'BSDAE_full_tmp_sqrt'

In [12]:
model_class = DAE

dense_nodes_encoder_choices = [[64],
                               [128],
                               [256],
                               [128,64],
                               [256,128],
                               [256,64],
                               [512,512,128],
                               [1024,1024,128],
                               [1024,1024,64],
                               [1024,1024,32],
                              ]

dense_nodes_decoder_choices = [[],
                               [],
                               [],
                               [128],
                               [256],
                               [256],
                               [512,512],
                               [1024,1024],
                               [1024,1024],
                               [1024,1024],
                              ]

In [13]:
def dnn_nodes_choice_to_txt(cnn_filters_choice):
    '''
    Returns cnn filters as text. 
    '''
    
    cnn_filter_txt  = str(cnn_filters_choice).replace('[', '').replace(']', '').replace(', ', '_')
    
    return cnn_filter_txt


def save_features(model_features,
                  model_id,
                  hyperparameter_index):
    with open('./hyperparameter-search-results/' + model_id + '_' +
              str(hyperparameter_index), 'wb+') as f:
        pickle.dump(model_features,f)
    return None

In [14]:

skf = StratifiedKFold(n_splits=5, random_state=5)
testing_errors = []
all_kf_errors = []


for dense_nodes_encoder_choice in dense_nodes_encoder_choices:

    network_id = dnn_nodes_choice_to_txt(dense_nodes_encoder_choice)

    model_features = dae_model_features(
        learining_rate=10**-3,
        l1_regularization_scale=0.0,
        dropout_probability=0.0,
        batch_size=32,
        output_size=1024,
        dense_nodes_encoder=dense_nodes_encoder_choice,
        dense_nodes_decoder=dense_nodes_encoder_choice[:-1],
        scaler=make_pipeline(FunctionTransformer(np.log1p, validate=True), Normalizer(norm='max')),
        activation_function=tf.nn.tanh,
        output_function=None)

    save_features(model_features,
              model_id,
              network_id)

    k_folds_errors = []



    model = model_class(model_features)
    optimizer = tf.train.AdamOptimizer(model_features.learining_rate)

    _, f1_error = model.fit_batch(
        (all_spectra, all_keys_modified),
        (all_spectra, all_keys_modified),
        optimizer=optimizer,
        num_epochs=5,
        verbose=1,
        obj_cost=model.mse,
        earlystop_cost_fn=model.mse,
        earlystop_patience=1000,
        data_augmentation=model.default_data_augmentation,
        augment_testing_data=False,
        print_errors=True,
        record_train_errors=False)


    model.save_weights('./hyperparameter-search-results/'+model_id+'_checkpoint_'+str(network_id)+'_',
                       save_format='tf')
    
    '''
    k = 0
    for train_index, test_index in skf.split(all_spectra, keys_raw):
        # reset model on each iteration
        model = model_class(model_features)
        optimizer = tf.train.AdamOptimizer(model_features.learining_rate)

        costfunction_errors_tmp, earlystop_errors_tmp = train_earlystop(
                training_data=all_spectra[train_index],
                training_keys=all_keys[train_index],
                testing_data=all_spectra[test_index],
                testing_keys=all_keys[test_index],
                model=model,
                optimizer=optimizer,
                num_epochs=20,
                obj_cost=model.mse,
                earlystop_cost_fn=model.mse,
                earlystop_patience=5,
                not_learning_patience=5,
                not_learning_threshold=1.0,
                verbose=True,
                fit_batch_verbose=1,
                data_augmentation=model.default_data_augmentation)
        k_folds_errors.append(earlystop_errors_tmp)
        model.save_weights('./hyperparameter-search-results/'+model_id+'_checkpoint_'+str(network_id)+'_'+str(k))

    all_kf_errors.append([k_folds_errors])

    testing_errors.append(np.average(k_folds_errors))
    np.save('./hyperparameter-search-results/final_test_errors_'+model_id, testing_errors)
    np.save('./hyperparameter-search-results/final_kf_errors_'+model_id, all_kf_errors)
    '''

INFO:tensorflow:Scale of 0 disables regularizer.
Epoch 1: CostFunc loss: 0.00 0.01, EarlyStop loss: 0.00 0.01
Epoch 2: CostFunc loss: 0.00 0.01, EarlyStop loss: 0.00 0.01
Epoch 3: CostFunc loss: 0.00 0.01, EarlyStop loss: 0.00 0.01
Epoch 4: CostFunc loss: 0.00 0.01, EarlyStop loss: 0.00 0.01
Epoch 5: CostFunc loss: 0.00 0.01, EarlyStop loss: 0.00 0.01
INFO:tensorflow:Scale of 0 disables regularizer.
Epoch 1: CostFunc loss: 0.00 0.01, EarlyStop loss: 0.00 0.01
Epoch 2: CostFunc loss: 0.00 0.01, EarlyStop loss: 0.00 0.01
Epoch 3: CostFunc loss: 0.00 0.01, EarlyStop loss: 0.00 0.01
Epoch 4: CostFunc loss: 0.00 0.01, EarlyStop loss: 0.00 0.01
Epoch 5: CostFunc loss: 0.00 0.01, EarlyStop loss: 0.00 0.01
INFO:tensorflow:Scale of 0 disables regularizer.
Epoch 1: CostFunc loss: 0.00 0.01, EarlyStop loss: 0.00 0.01
Epoch 2: CostFunc loss: 0.00 0.01, EarlyStop loss: 0.00 0.01
Epoch 3: CostFunc loss: 0.00 0.01, EarlyStop loss: 0.00 0.01
Epoch 4: CostFunc loss: 0.00 0.01, EarlyStop loss: 0.00 0.01